# Analysis of Historical NYC Sales Data 2003-2018

In [4]:
# Import required analysis libraries
import numpy as np
import pandas as pd
import csv

In [41]:
# Import required visualization libraries
%matplotlib inline
import hvplot.pandas
import plotly.express as px
from panel.interact import interact

### Importing Borough sales data, pre-processed using SQL

In [35]:
manhattan_path = 'Resources/manhattan_sales.csv'
manhattan_df = pd.read_csv(manhattan_path)
manhattan_df.head()

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BLOCK,LOT,ZIPCODE,SALE PRICE,SALE DATE
0,ALPHABET CITY,02 TWO FAMILY HOMES,375,32,10009,1800000.0,2003-01-22 00:00:00
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,372,31,10009,0.0,2003-12-18 00:00:00
2,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,372,31,10009,0.0,2003-12-18 00:00:00
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,378,33,10009,426000.0,2003-10-23 00:00:00
4,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,391,12,10009,0.0,2003-02-28 00:00:00


In [36]:
# remove irrelevant columns
manhattan_df.rename(columns={'SALE DATE':'SALEDATE'}, inplace=True)
manhattan_df['Year'] = manhattan_df.SALEDATE.str[:4]
manhattan_df['NEIGHBORHOOD']=manhattan_df['NEIGHBORHOOD'].str.strip()
manhattan_df.drop(columns=['BLOCK','LOT','ZIPCODE','SALEDATE'], inplace=True)

# filter out potential transfers, and not sales, define new dataframe
manhattan_sales_df = manhattan_df[manhattan_df['SALE PRICE']>5000]
print(manhattan_sales_df.describe())
manhattan_sales_df.head()


         SALE PRICE
count  2.907760e+05
mean   2.590291e+06
std    2.585381e+07
min    5.020000e+03
25%    3.830000e+05
50%    7.331400e+05
75%    1.550000e+06
max    4.111112e+09


,NEIGHBORHOOD,BUILDING CLASS CATEGORY,SALE PRICE,Year
0,ALPHABET CITY,02 TWO FAMILY HOMES,1800000.0,2003
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,426000.0,2003
11,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,1600000.0,2003
12,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,1060000.0,2003
15,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,3200000.0,2003


In [40]:
mttn_neigh_sales = manhattan_sales_df.groupby(['NEIGHBORHOOD','Year']).sum().reset_index()
manhattan_neighborhoods = set(mttn_neigh_sales['NEIGHBORHOOD'])
mttn_neigh_sales.head()

,NEIGHBORHOOD,Year,SALE PRICE
0,ALPHABET CITY,2003,58695200.0
1,ALPHABET CITY,2004,94969162.0
2,ALPHABET CITY,2005,159132304.0
3,ALPHABET CITY,2006,143839304.0
4,ALPHABET CITY,2007,221430397.0


## Manhattan Real Estate Sales by Neighbhorhood (2003 -2018)

In [45]:
def mttn_neigh_sales_plot(Neighborhood):
    mttn_slice = mttn_neigh_sales[mttn_neigh_sales['NEIGHBORHOOD'].isin([Neighborhood])]
    return mttn_slice.hvplot.line(
        x='Year',
        y='SALE PRICE',
        title='Manhattan Real Estate Sales'
        ).opts(xlabel='Year', ylabel='Total Sales',yformatter="%.0f")
interact(mttn_neigh_sales_plot, Neighborhood=manhattan_neighborhoods)

Column
    [0] Column
        [0] Select(name='Neighborhood', options=['UPPER EAST SIDE (59-79)'...], value='UPPER EAST SIDE (59-79)')
    [1] Row
        [0] HoloViews(Curve, name='interactive01895')

## Rate of Change and Standard Deviation of Rate of Change by Neighborhood

In [ ]:
for 